<a href="https://colab.research.google.com/github/asandy1023/python-demo/blob/main/taiwan_aicup_Spring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

taiwan_aicup_Spring



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math


class PositionalEncoding(nn.Module):
    def __init__(self, d_emb: int, dropout: float = 0.1, max_len: int = 200):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        self.pe = torch.zeros(max_len, d_emb)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_emb, 2) *
                             -(math.log(10000.0) / d_emb))
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = self.pe.unsqueeze(0)

    def forward(self, src):
        pe = self.pe.detach().to(src.device)
        output = src + pe[:, :src.size(1)]
        return self.dropout(output)


class MultiHeadAttention(nn.Module):
    def __init__(self, d_hid, n_head):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.d_k = d_hid // n_head
        self.query = nn.Parameter(torch.randn(1, d_hid))
        self.key = nn.Linear(d_hid, d_hid)
        self.value = nn.Linear(d_hid, d_hid)
        self.linear = nn.Linear(d_hid, d_hid)

    def forward(self, x, batch_tk_mask):
        B = x.size(0)

        # Input  shape: `(1, Hid)`.
        # Output shape: `(Head, 1, K)`.
        q = self.query.view(-1, self.n_head, self.d_k).transpose(0, 1)

        # Transform temporal features to query, key and value features.
        # Input  shape: `(B, S, Hid)`.
        # Output shape: `(B, Head, S, K)`.
        k = self.key(x).view(B, -1, self.n_head, self.d_k).transpose(1, 2)
        v = self.value(x).view(B, -1, self.n_head, self.d_k).transpose(1, 2)

        # Calculate self attention scores with query and key features.
        # Self attention scores are scaled down by hidden dimension square
        # root to avoid overflow.
        # `q` Input  shape: `(Head, 1, K)`.
        # `k` Input  shape: `(B, Head, S, K)`.
        # Output shape: `(B, Head, 1, S)`.
        # print(q.shape)
        # print(k.shape)
        attn = q @ k.transpose(-1, -2) / math.sqrt(x.size(-1))

        # Mask parts of attention scores by replacing with large negative
        # values.
        # Input  shape: `(B, Head, 1, S)`.
        # Output shape: `(B, Head, 1, S)`.
        batch_tk_mask = batch_tk_mask.repeat(self.n_head, 1, 1, 1)

        batch_tk_mask = batch_tk_mask.transpose(0, 1)
        batch_tk_mask = batch_tk_mask.transpose(-1, -2)
        # print(attn.shape)
        # print(batch_tk_mask.shape)
        attn.masked_fill_(batch_tk_mask, -1e9)

        # Softmax normalize on attention scores.
        # Large negative values will be closed to zero after normalization.
        # Input  shape: `(B, Head, 1, S)`.
        # Output shape: `(B, Head, 1, S)`.
        attn = F.softmax(attn, dim=-1)

        # Use attention scores to calculate weighted sum on value features.
        # Then perform one more linear tranformation on weighted sum.
        # Finally dropout transformed features.
        # `attn` Input  shape: `(B, Head, 1, S)`.
        # `v` Input  shape: `(B, Head, S, k)`.
        # Output shape: `(B, Head, 1, K)`.
        output = attn @ v

        # Input  shape: `(B, Head, 1, K)`.
        # Output shape: `(B, 1, Hid)`.
        output = output.transpose(1, 2).contiguous()
        output = output.view(B, -1, self.n_head * self.d_k)

        # Output shape: `(B, Hid)`.
        return self.linear(output.squeeze(1))


class Encoder(nn.Module):
    def __init__(self, d_emb: int, p_hid: float):
        super().__init__()
        self.linear = nn.Linear(d_emb, d_emb)
        self.pe = PositionalEncoding(d_emb, p_hid)
        self.attn_emb = MultiHeadAttention(d_emb, 4)
        self.layernorm1 = nn.LayerNorm(d_emb)
        self.layernorm2 = nn.LayerNorm(d_emb)

    def forward(self, x: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        # Shape: [B, S, H]
        emb = self.layernorm1(self.linear(x))

        # Shape: [B, S, H]
        emb = self.pe(emb)

        # Shape: [B, H]
        emb = self.layernorm2(self.attn_emb(emb, mask))

        return emb


# Risk Classificaion

## Take a look at the risk classification dataset

In [ ]:
import numpy as np
import csv
import json
import unicodedata
import re
from torch.utils.data import Dataset, DataLoader

'''
Here we will do preprocessing on the dataset.
Something needs to be done here :
1. Read the file in.
2. Separate the article, question, answer.
3. Used PAD to round each sentence into the same length
'''


def split_sent(sentence: str):
    first_role_idx = re.search(':', sentence).end(0)
    out = [sentence[:first_role_idx]]

    tmp = sentence[first_role_idx:]
    while tmp:
        res = re.search(
            r'(護理師[\w*]\s*:|醫師\s*:|民眾\s*:|家屬[\w*]\s*:|個管師\s*:)', tmp)
        if res is None:
            break

        idx = res.start(0)
        idx_end = res.end(0)
        out[-1] = list(out[-1] + tmp[:idx])
        out.append(tmp[idx:idx_end])
        tmp = tmp[idx_end:]

    out[-1] = list(out[-1] + tmp)

    return out

def _read_risk(risk_file: str):
    article = []
    risk = []

    # [[Sent_1], [Sent_2], ..., [Sent_n]]
    for i, line in enumerate(csv.reader(open(risk_file, "r", encoding="utf-8"))):
        if i == 0:
            continue
        text = unicodedata.normalize("NFKC", line[2]).replace(" ", "")
        article.append(split_sent(text))
        #risk.append(int(line[3]))
        risk.append(line[3])

    return article, risk

def _read_qa(qa_file: str):
    qa = {}
    # [Question, [[Choice_1, Answer_1], [Choice_2, Answer_2], [Choice_3, Answer_3]]]
    for data in json.loads(unicodedata.normalize("NFKC", open(qa_file, "r", encoding="utf-8").read())):
        question = data["question"]
        choice_ans = [(
                list(choice["text"]),
                int(choice["label"].strip() == data["answer"].strip())
            ) for choice in question["choices"]]
        question_text = list(question["stem"])
        aid = data["article_id"]
        if aid in qa:
            qa[aid][1].append((question_text, choice_ans))
        else:
            qa[aid] = (split_sent(data['text'].replace(" ", "")), [(question_text, choice_ans)])

    return zip(*[v for _,v in sorted(qa.items(),key=lambda x:x[0])])


def _read_qa1(qa_file: str):
    qa = {}
    # [Question, [[Choice_1, Answer_1], [Choice_2, Answer_2], [Choice_3, Answer_3]]]
    for data in json.loads(unicodedata.normalize("NFKC", open(qa_file, "r", encoding="utf-8").read())):
        question = data["question"]
        choice_ans = [(
                list(choice["text"]),
                int(choice["label"].strip() == "A")
            ) for choice in question["choices"]]
        question_text = list(question["stem"])
        aid = data["article_id"]
        if aid in qa:
            qa[aid][1].append((question_text, choice_ans))
        else:
            qa[aid] = (split_sent(data['text'].replace(" ", "")), [(question_text, choice_ans)])

    return zip(*[v for _,v in sorted(qa.items(),key=lambda x:x[0])])


def encode_sent(w2id: dict, sentence: list, max_length: int):
    output = []
    for i, token in enumerate(sentence):
        if i >= max_length:
            break
        if token in w2id:
            output.append(w2id[token])
        else:
            output.append(0)
    padding_word = [0]
    sent_padding_size = max_length - len(output)
    output = output + padding_word*sent_padding_size

    return output


def encode_articles(article_text, max_doc_len, w2id, max_sent_len):
    article = []
    for document in article_text:
        article.append([])
        for i, sentence in enumerate(document):
            if i >= max_doc_len:
                break
            article[-1].append([])
            article[-1][-1] = encode_sent(w2id, sentence, max_sent_len)
        padding_sent = [[0]*max_sent_len]
        doc_padding_size = max_doc_len - len(article[-1])
        article[-1] = article[-1] + padding_sent*doc_padding_size
    return np.array(article)



class dataset_qa1(Dataset):
    def __init__(
        self,
        vocab_path: str,
        qa_file: str,
        max_sent_len: int = 52,
        max_doc_len: int = 170,
        max_q_len: int = 20,
        max_c_len: int = 18
    ):
        super().__init__()
        with open(vocab_path, 'r', encoding='utf-8') as f_w2id:
            w2id = json.load(f_w2id)

        article_text, qa_pairs = _read_qa1(qa_file)

        # `article` shape: [N, `max_doc_len`, `max_sent_len`]
        self.article = encode_articles(
            article_text, max_doc_len, w2id, max_sent_len)

        self.QA = []
        for idx, qa_pair in enumerate(qa_pairs):
            for question, choice_ans in qa_pair:
                choice, ans = zip(*choice_ans)
                self.QA.append({
                    "article": self.article[idx],
                    "question": np.array(encode_sent(w2id, question, max_q_len)),
                    "choice": np.array([encode_sent(w2id, x, max_c_len) for x in choice]),
                    "qa_answer": np.array(ans),
                })

    def __len__(self):
        return len(self.QA)

    def __getitem__(self, idx: int):
        return self.QA[idx]

class dataset_qa(Dataset):
    def __init__(
        self,
        vocab_path: str,
        qa_file: str,
        max_sent_len: int = 52,
        max_doc_len: int = 170,
        max_q_len: int = 20,
        max_c_len: int = 18
    ):
        super().__init__()
        with open(vocab_path, 'r', encoding='utf-8') as f_w2id:
            w2id = json.load(f_w2id)

        article_text, qa_pairs = _read_qa(qa_file)

        # `article` shape: [N, `max_doc_len`, `max_sent_len`]
        self.article = encode_articles(
            article_text, max_doc_len, w2id, max_sent_len)

        self.QA = []
        for idx, qa_pair in enumerate(qa_pairs):
            for question, choice_ans in qa_pair:
                choice, ans = zip(*choice_ans)
                self.QA.append({
                    "article": self.article[idx],
                    "question": np.array(encode_sent(w2id, question, max_q_len)),
                    "choice": np.array([encode_sent(w2id, x, max_c_len) for x in choice]),
                    "qa_answer": np.array(ans),
                })

    def __len__(self):
        return len(self.QA)

    def __getitem__(self, idx: int):
        return self.QA[idx]


class dataset_risk(Dataset):
    def __init__(
        self,
        vocab_path: str,
        risk_file: str,
        max_sent_len: int = 52,
        max_doc_len: int = 170,
    ):
        super().__init__()
        with open(vocab_path, 'r', encoding='utf-8') as f_w2id:
            w2id = json.load(f_w2id)
        # w2id = {"[PAD]": 0}
        article_text, risk = _read_risk(risk_file)

        # `risk` shape: [N]
        #self.risk = np.array(risk, dtype=np.float32)
        self.risk = np.array(risk)

        # `article` shape: [N, `max_doc_len`, `max_sent_len`]
        self.article = encode_articles(
            article_text, max_doc_len, w2id, max_sent_len)

    def __len__(self):
        return len(self.risk)

    def __getitem__(self, idx: int):
        return {"article": self.article[idx], "risk_answer": self.risk[idx]}


In [ ]:
import os
import pandas as pd
import json
#from dataset import dataset_risk

risk_file=os.path.join("data", "Train_risk_classification_ans.csv")
#risk_file=os.path.join("data", "Develop_risk_classification.csv")
print('risk data:')
print(pd.read_csv(risk_file,encoding='utf-8',usecols=['article_id','text','label']))
print('\n---------------------------------')
print('vocab:')
print(list(json.load(open(os.path.join("data", "vocab.json"),encoding='utf-8')).items())[:10])

dataset = dataset_risk(
    vocab_path=os.path.join("data", "vocab.json"),
    risk_file=risk_file,
)
d = next(iter(dataset))
print('\n---------------------------------')
print('text\n', d['article'])
print(d['article'].shape)
r = {v: k for k, v in json.load(open(os.path.join("data", "vocab.json"),encoding='utf-8')).items()}
print(*[''.join(map(lambda x:r[x], i))[:40] for i in d['article'][:4,:] ], sep='\n')
print('\n---------------------------------')
print('answer:\n', d['risk_answer'])

risk data:
     article_id                                               text label
0             1  個管師：這個月還好嗎？民眾：蛤？個管師：這個月還好嗎？民眾：這個月還好。個管師：還好，還可以...     1
1             2  個管師：所以你這個月還OK？民眾：還OK啊。個管師：那有固定伴侶嗎？民眾：沒有。個管師：你一...     1
2             3  民眾：啊就在忙裝潢啊～幹嘛的。個管師：嗯。民眾：對。個管師：所以都在熬夜？民眾：對，都一直熬...     1
3             4  個管師：對啊，你是說你要幫，我幫你給這個？你說小花跟你。民眾：現在是狀況是還好。個管師：嗯。...     1
4             5  個管師：好喔，過去這一個月還好嗎？民眾：恩，還可以。個管師：那今天是請假？民眾：沒有，晚上上...     0
..          ...                                                ...   ...
341         342  醫師：好，謝謝。等一下結束後，科技部會給你兩百五十塊的車馬費。民眾：車馬費？醫師：對啦，就是...     1
342         343  醫師：就照舊這樣子。阿這一定要跟你講一下。家屬：沒關係啊。醫師：阿最近好嗎？民眾：都正常。醫...     0
343         344  醫師：好。你這次的抽血都還不錯喔。民眾：嗯。醫師：看一下，應該是年度的檢查，我們下次就，就一...     0
344         345  民眾：我還要問……醫師：是。民眾：問醫師一個問題，我的肝藥喔。醫師：誒。民眾：你給我開兩個月...     0
345         346  醫師：阿現在是在哪，在，也是在念碩班？民眾：對阿。醫師：要畢業了嗎？民眾：希望是快了。醫師：...     0

[346 rows x 3 columns]

---------------------------------
vocab:
[('[PAD]', 0), ('，', 1), ('的', 2), ('。', 3), ('

In [ ]:
import sys
np.load('data/embeddings.npy')

array([[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [ 0.008 ,  0.0336,  0.572 , ..., -0.0106,  0.0009, -0.0085],
       [-0.0119,  0.051 ,  0.4397, ..., -0.0585,  0.0369, -0.027 ],
       ...,
       [ 0.0895, -0.1197, -0.0107, ..., -0.0607,  0.0213,  0.0018],
       [ 0.1285,  0.0442,  0.1356, ...,  0.0439,  0.0114, -0.1839],
       [-0.0441, -0.0893,  0.0644, ..., -0.0484, -0.0489, -0.0562]])

In [ ]:
print(len(tt[1051]))

300


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.zh.300.vec.gz

--2021-06-13 11:05:13--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.zh.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1358817100 (1.3G) [binary/octet-stream]
Saving to: ‘cc.zh.300.vec.gz’

cc.zh.300.vec.gz    100%[===================>]   1.26G  39.6MB/s    in 34s     

2021-06-13 11:05:47 (38.5 MB/s) - ‘cc.zh.300.vec.gz’ saved [1358817100/1358817100]



In [ ]:
!gunzip cc.zh.300.vec.gz

In [ ]:
import json
tt=open('data/vocab.json')

In [ ]:
import numpy as np
import csv
import json
import unicodedata
import re
import time
import random
import os
from tqdm import tqdm


'''
Here we will convert the word vector into a readable form, and generate two file and a path: 
1. vocab.json     : Used to record the index of each text
2. embeddings.npy : Used to record the word vector in npy form
3. ./data         : Used to store the generated data, and our dataset in here
'''

def load_vectors(fname):


    if os.path.isdir("./data"):
        pass
    else:
        os.makedirs("./data")

    with open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore') as WordEmbeddings,\
            open("data/vocab.json", 'w', encoding='utf-8') as vocab_write:
        wv = []
        vocab = {}
        vocab["[PAD]"] = 0
        wv.append(np.zeros(300))
        for i, line in enumerate(WordEmbeddings):
            word, vec = line.split(' ', 1)
            if i == 0:
                continue
            vocab[word] = i
            wv.append(np.fromstring(vec, sep=' '))
        json.dump(vocab, vocab_write, ensure_ascii=False)

    np.save("data/embeddings.npy", np.array(wv))

'''
Put your word vector path here.
In this baseline, our word vector is to use fasttext.
You can find it in here. (https://fasttext.cc/docs/en/crawl-vectors.html)
'''

if __name__ == "__main__":
    load_vectors("cc.zh.300.vec")

## Risk Classification model

In [ ]:
class Risk_Classifier(nn.Module):
    def __init__(self, d_emb: int, p_drop: float, n_layers: int):
        super().__init__()
        hid = []
        self.l0 = nn.Linear(d_emb, d_emb)
        for _ in range(n_layers):
            hid.append(nn.Linear(in_features=d_emb, out_features=d_emb))
            hid.append(nn.ReLU())
            hid.append(nn.Dropout(p=p_drop))
        self.hid = nn.Sequential(*hid)
        self.l1 = nn.Linear(d_emb, d_emb//2)
        self.dropout = nn.Dropout(p_drop)
        self.l2 = nn.Linear(d_emb//2, 1)

    def forward(self, document: torch.Tensor) -> torch.Tensor:
        output = document
        output = self.l0(output)

        output = self.hid(output)
        # Linear layer
        # Input shape: `(B, E)`
        # Ouput shape: `(B, E//2)`
        # output = F.relu(self.l1(document))
        output = F.relu(self.l1(output))

        # Dropout
        # Input shape: `(B, E//2)`
        # Ouput shape: `(B, E//2)`
        # output = self.dropout(output)

        # Linear layer
        # Input shape: `(B, E//2)`
        # Ouput shape: `(B, 1)`
        output = torch.sigmoid(self.l2(output))

        return output.squeeze(-1)


class risk_model(nn.Module):
    def __init__(self, embedding_path: str, d_emb: int, n_cls_layers: int, p_drop: float):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.cuda.FloatTensor(
            np.load(embedding_path)), freeze=True, padding_idx=0)
        self.word_encoder = Encoder(d_emb, p_drop)
        self.encoder = Encoder(d_emb, p_drop)
        self.risk = Risk_Classifier(d_emb, p_drop, n_cls_layers)

    def forward(self, document):
        # Embedding layer
        # Shape: [B, `max_doc_len`, `max_sent_len`, E]
        doc = self.embedding(document)
        w_mask, s_mask = self.create_mask(document)

        # Sentence embedding
        # Input shape: [B, `max_doc_len`, `max_sent_len`, E]
        # Output shape: [B, `max_doc_len`, E]
        doc = torch.stack([ self.word_encoder(d,w) for d,w in zip(doc, w_mask)])

        # Document embedding
        # Input shape: [B, `max_doc_len`, E]
        # Output shape: [B, E]
        doc = self.encoder(doc, s_mask)

        risk_output = self.risk(doc)

        return risk_output

    @staticmethod
    def create_mask(batch_prev_tkids: torch.Tensor) -> torch.Tensor:
        # Create padding self attention masks.
        # Shape: [B, `max_doc_len`, `max_sent_len`, 1]
        # Output dtype: `torch.bool`.
        w_pad_mask = batch_prev_tkids == 0
        w_pad_mask = w_pad_mask.unsqueeze(-1)

        s_pad_mask = batch_prev_tkids.sum(dim=-1)
        s_pad_mask = s_pad_mask == 0
        s_pad_mask = s_pad_mask.unsqueeze(-1)

        return w_pad_mask, s_pad_mask

    def loss_fn(self, document, risk):
        pred_risk = self(document)
        pred_risk = pred_risk.reshape(-1)
        risk = risk.reshape(-1)
        return F.binary_cross_entropy(pred_risk, risk)


## Training Risk Classifier

In [ ]:
import csv
import os
import pathlib
import random
import re
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score

def risk_train(model_cfg, dataset, device,  # model and datasets
               p_drop, n_epoch, batch_size, learning_rate,  # training hyper parameter
               save_step, model_path):  # saving model

    model = risk_model(**model_cfg, p_drop=p_drop).train().to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    dataldr = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    def save_model(md, step):
        torch.save(md.state_dict(), os.path.join(
            model_path, f"model-{step}.pt"))

    # Train loop
    step = 0
    for epoch in range(n_epoch):
        tqdm_dldr = tqdm(dataldr)

        avg_loss = 0
        for epoch_step, batch_data in enumerate(tqdm_dldr):
            optimizer.zero_grad()

            batch_document = torch.Tensor(np.array(batch_data["article"], dtype='float32')).to(device)
            batch_document = batch_document.to(torch.int64)
            batch_risk = torch.Tensor(np.array(batch_data["risk_answer"], dtype='float32')).to(device)
            batch_risk = batch_risk.float()
            
            loss = model.loss_fn(batch_document, batch_risk)
            loss.backward()
            optimizer.step()

            step += 1
            avg_loss += loss
            tqdm_dldr.set_description(
                f"epoch:{epoch}, loss:{avg_loss / (epoch_step+1):.04f}")

            if step % save_step == 0:
                save_model(model, step)

    save_model(model, step)





random_seed = 42
# Set random states for reproducibility
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
# Use cuda when possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Save training configuration
risk_model_path = os.path.join("exp", "risk")
pathlib.Path(risk_model_path).mkdir(parents=True, exist_ok=True)


model_cfg = {
    "embedding_path": os.path.join("data", "embeddings.npy"),
    "d_emb": 300,
    "n_cls_layers": 2,
}

dataset = dataset_risk(
    vocab_path=os.path.join("data", "vocab.json"),
    risk_file=os.path.join("data", "Train_risk_classification_ans.csv"),
)

risk_train(
    model_cfg=model_cfg,
    dataset=dataset,
    model_path=risk_model_path,
    device=device,
    # Hyperparameters
    batch_size=2,
    learning_rate=1e-4,
    n_epoch=120,
    save_step=80,
    p_drop=0.1,
)



## Test on trained model

In [ ]:
def save_result(output_path: str, data: list, ckpt: int):
    output = [["id", "label"]] + \
        [[i+1, label] for i, label in enumerate(data)]
    csv.writer(open(os.path.join(
        output_path, f"decision_{ckpt}.csv"), 'w', newline='')).writerows(output)
        
@torch.no_grad()
def risk_test(model_cfg, dataset, device, batch_size,
              model_path, output_path):
    dataldr = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=False)

    # Load all checkpoints
    ckpts = sorted([
        (int(ckpt.group(1)), os.path.join(risk_model_path, ckpt.group(0)))
        for ckpt in map(lambda f:re.match(r'model-(\d+).pt', f), os.listdir(model_path))
        if ckpt is not None
    ], key=lambda x: x[0])

    for step, ckpt in ckpts:
        # Model
        model = risk_model(**model_cfg, p_drop=0.0)
        model.load_state_dict(torch.load(ckpt))
        model = model.eval().to(device)

        preds = []

        for batch_data in tqdm(dataldr):
            batch_document = batch_data["article"].to(device)
            batch_document = batch_document.to(torch.int64)
            preds += model(batch_document).tolist()
#         print(f"\nroc_auc {step} : {roc_auc_score(dataset.risk, preds):.04f}", flush=True)
        
        save_result(output_path, preds, step)

risk_output_path = os.path.join("output", "risk")
pathlib.Path(risk_output_path).mkdir(parents=True, exist_ok=True)

print("\nevaluate on training set...", flush=True)
risk_test(
    model_cfg=model_cfg,
    dataset=dataset,
    model_path=risk_model_path,
    device=device,
    batch_size=4,
    output_path=risk_output_path,
)



evaluate on training set...


100%|██████████| 87/87 [00:01<00:00, 85.25it/s]


## Preditions

In [ ]:
import pandas as pd
risk_latest_result = sorted([
    (int(ckpt.group(1)), os.path.join(risk_output_path, ckpt.group(0)))
    for ckpt in map(lambda f:re.match(r'decision_(\d+).csv', f), os.listdir(risk_output_path))
    if ckpt is not None
], key=lambda x: x[0])[-1]

pd.read_csv(open(risk_latest_result[1]))

,id,label
0,1,0.999851
1,2,0.999819
2,3,0.999871
3,4,0.999906
4,5,0.001771
...,...,...
341,342,0.999381
342,343,0.000144
343,344,0.000209
344,345,0.002352


In [ ]:
import csv
ans_file = open('data/Train_risk_classification_ans.csv')
decision_file = open('output/decision_8400.csv')
rows1 = csv.reader(ans_file)
rows2 = csv.reader(decision_file)
i1=0
i2=0
row2_e=""
row1_e=""
row2_i=0
row1_i=0
gate=False
for row1,row2 in zip(rows1,rows2):
  if gate==False:
    gate=True
    continue
  else:
    row2_e=row2[1].strip()
    if int(row2_e[0])>0:
      row2_i=0
    else:
      if int(row2_e[2])>5:
        row2_i=1
      else:
        row2_i=0
    row1_e=row1[3].strip()
    row1_i=int(row1_e[0])
    print("正解:",row1_e,"int(正解:):",row1_i,"int(算出的:",row2_i,"算出的:",row2_e,sep=' ')
    if row1_i==row2_i:
      i2=i2+1
    else:
      print("============Error item============",sep='')
    i1=i1+1
print("完成機率:",i2/i1)

正解: 1 int(正解:): 1 int(算出的: 1 算出的: 0.9998506307601929
正解: 1 int(正解:): 1 int(算出的: 1 算出的: 0.9998193383216858
正解: 1 int(正解:): 1 int(算出的: 1 算出的: 0.9998712539672852
正解: 1 int(正解:): 1 int(算出的: 1 算出的: 0.9999055862426758
正解: 0 int(正解:): 0 int(算出的: 0 算出的: 0.0017706301296129823
正解: 0 int(正解:): 0 int(算出的: 1 算出的: 0.8109180331230164
============Error item============
正解: 1 int(正解:): 1 int(算出的: 1 算出的: 0.9998190999031067
正解: 0 int(正解:): 0 int(算出的: 0 算出的: 0.11472171545028687
正解: 0 int(正解:): 0 int(算出的: 0 算出的: 0.02365202084183693
正解: 0 int(正解:): 0 int(算出的: 0 算出的: 0.0030509098432958126
正解: 1 int(正解:): 1 int(算出的: 1 算出的: 0.9997724890708923
正解: 1 int(正解:): 1 int(算出的: 1 算出的: 0.9998962879180908
正解: １ int(正解:): 1 int(算出的: 1 算出的: 0.9997734427452087
正解: 1 int(正解:): 1 int(算出的: 1 算出的: 0.9998459815979004
正解: 1 int(正解:): 1 int(算出的: 1 算出的: 0.9998270869255066
正解: １ int(正解:): 1 int(算出的: 1 算出的: 0.9999028444290161
正解: 1 int(正解:): 1 int(算出的: 1 算出的: 0.9998565912246704
正解: 0 int(正解:): 0 int(算出的: 0 算出的: 0.0006542004412040114


# Question Answer

## About Question Answer dataset

In [ ]:
from pprint import pprint
#from dataset import dataset_qa,dataset_qa1
import os
import pandas as pd
import json

#qa_file=os.path.join("data", "SampleData_QA.json")
qa_file=os.path.join("data", "Develop_QA.json")

print('qa data:')
print(json.load(open(qa_file,encoding='utf-8'))[0])

dataset = dataset_qa(
    vocab_path=os.path.join("data", "vocab.json"),
    qa_file=os.path.join("data", "Train_qa_ans.json"),
)
#dataset = dataset_qa1(
#    vocab_path=os.path.join("data", "vocab.json"),
#    qa_file=os.path.join("data", "Train_qa_ans.json"),
#)

d = next(iter(dataset))
print('\n---------------------------------')
print('encoded article')
print(d['article'])
print(d['article'].shape)
print('\n---------------------------------')
print('encoded question')
print(d['question'])
print(d['question'].shape)
print('\n---------------------------------')
print('encoded choice')
print(d['choice'])
print(d['choice'].shape)
print('\n---------------------------------')
print('answer')
print(d['qa_answer'])

qa data:
{'id': 1, 'article_id': 1, 'text': '個管師：好喔，我剛剛後來確認了一下你抽血的狀況，哈哈～對，你下一次抽血就10月20到11月1號。民眾：好。個管師：然後廖醫師有幫你排回診2月17號。民眾：好。個管師：這樣ok嗎？民眾：好。個管師：然後你現階段……民眾：是。個管師：你跟你男朋友都沒有約嗎？民眾：沒有啊。個管師：然後這一個月沒有性行為？民眾：也沒有阿。個管師：你們沒有住在一起對不對？民眾：有。個管師：你們有住在一起，你們有住在一起，但是性行為可以預期嗎？民眾：蛤，可以啊。個管師：呵～真的喔？民眾：一個，因為我看，我是看著一個就是現實社會的狀況比那個更重的人，所以……個管師：什麼意思？民眾：就是，因為他現在都是工讀阿，所以他現在打了兩份工，所以那個時間，然後所以時間到我一定逼他去工作，呵呵～個管師：喔他現在在，誒他現在在工作了？民眾：他現在是在工作啦。個管師：他剛畢業嗎？民眾：沒有阿。個管師：畢業兩年？民眾：一段時間了。個管師：一年？民眾：已經超過了，對然後……個管師：那位什麼還在考license？民眾：阿之前都沒過啊。個管師：那他還這麼堅持，還要考license喔？民眾：是，我後來他遇到我之後有跟我講到這件事情，然後我就想說好，你要……個管師：那他有興趣在臨床嗎？民眾：不知道誒，但是他有想要說先考過再說，那我就說好，那你就先去看說，我現在就是讓他自己，不打擾，然後你就工作看書，就是對。個管師：喔，所以你們的性行為是可以被預期的？民眾：可以啊。個管師：好，因為通常……民眾：而且有加上這件事情就是他現在應該很會，很怕看到我這件事情～個管師：為什麼？民眾：看書。個管師：誒你這樣給他壓力太大了好不好。民眾：呵呵～個管師：你這樣給他壓力太大了啦。民眾：誒，你也要想想看你已經，就是對一般人來講你如果已經出來社會之後你會多少，記住多少學校教的事情？個管師：然後呢？民眾：對阿所以你要就是想辦法要重新再預習，而且你出社會經過兩三年過後我相信教的東西一定會比原本的……個管師：差很多？民眾：差很多啊。個管師：現在差很多？民眾：對，所以我還去補，跟認識的朋友那邊就是，他那邊之前有……個管師：拿書？民眾：補習，對，就是跟他借教材。個管師：恩，你對他真的蠻用心的，好啦，因為通常住在一起的話我們通常都會建議每天吃啦。民眾：恩。個管師：那如果假設

## Question Answer Model

In [ ]:
class QA_Classifier(nn.Module):
    def __init__(self, d_emb: int, p_hid: float, n_layers: int):
        super().__init__()
        self.l1 = nn.Linear(3*d_emb, d_emb)
        self.dropout = nn.Dropout(p_hid)

        hid = []
        for _ in range(n_layers):
            hid.append(nn.Linear(in_features=d_emb, out_features=d_emb))
            hid.append(nn.ReLU())
            hid.append(nn.Dropout(p=p_hid))
        self.hid = nn.Sequential(*hid)
        self.l2 = nn.Linear(d_emb, 1)

    def forward(
        self,
        document: torch.Tensor,
        question: torch.Tensor,
        choice: torch.Tensor
    ) -> torch.Tensor:
        # Concatenates `document embedding`, `question embedding`
        # and `choice embeding`
        # Input shape: `(B, E)`, `(B, E)`, `(B, E)`
        # Ouput shape: `(B, 3*E)`
        output = torch.cat((document, question, choice), -1)

        # Linear layer
        # Input shape: `(B, 3*E)`
        # Ouput shape: `(B, E)`
        output = F.relu(self.l1(output))

        # Dropout
        # Input shape: `(B, E)`
        # Ouput shape: `(B, E)`
        output = self.dropout(output)

        # Hidden layer
        output = self.hid(output)

        # Linear layer
        # Input shape: `(B, E)`
        # Ouput shape: `(B, 1)`
        output = torch.sigmoid(self.l2(output))

        return output


class qa_model(nn.Module):
    def __init__(self, embedding_path: str, d_emb: int, n_cls_layers: int, p_drop: float):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(
            np.load(embedding_path)), freeze=True, padding_idx=0)
        self.word_encoder = Encoder(d_emb, p_drop)
        self.encoder = Encoder(d_emb, p_drop)
        self.qa = QA_Classifier(d_emb, p_drop, n_cls_layers)

    def forward(self, document, question, choice):
        # Embedding layer
        # Shape: [B, `max_doc_len`, `max_sent_len`, E]
        doc = self.embedding(document)
        # Shape: [B, `max_q_len`, E]
        qst = self.embedding(question)
        # Shape: [B, 3, `max_c_len`, E]
        chs = self.embedding(choice)

        # Sentence embedding
        # Shape: [B, `max_doc_len`, E]
        w_mask, s_mask = self.create_mask(document)
        
        doc = torch.stack([self.word_encoder(d, m) for d, m in zip(doc, w_mask)])

        # Shape: [B, E]
        w_mask, _ = self.create_mask(question)
        qst = self.word_encoder(qst, w_mask)

        # Document embedding
        # Input shape: [B, `max_doc_len`, E]
        # Output shape: [B, E]
        doc = self.encoder(doc, s_mask)

        # Input Shape: [3, B, E]
        # Output Shape: [[B],[B],[B]]
        chs = chs.transpose(0, 1)
        w_mask, _ = self.create_mask(choice.transpose(0, 1))
        qa_output = [self.qa(doc, qst, self.word_encoder(ci, wmi)) for ci, wmi in zip(chs, w_mask)]
        qa_output = torch.cat(qa_output, dim=-1)
        return qa_output

    def create_mask(self, batch_prev_tkids: torch.Tensor) -> torch.Tensor:
        # Create padding self attention masks.
        # Shape: [B, `max_doc_len`, `max_sent_len`, 1]
        # Output dtype: `torch.bool`.
        w_pad_mask = batch_prev_tkids == 0
        w_pad_mask = w_pad_mask.unsqueeze(-1)

        s_pad_mask = batch_prev_tkids.sum(dim=-1)
        s_pad_mask = s_pad_mask == 0
        s_pad_mask = s_pad_mask.unsqueeze(-1)

        return w_pad_mask, s_pad_mask

    def loss_fn(self, document, question, choice, qa):
        question = torch.tensor(question).to(device).long()
        choice = torch.tensor(choice).to(device).long()
        pred_qa = self(document, question, choice)
        pred_qa = pred_qa.reshape(-1)
        qa = qa.reshape(-1)
        return F.binary_cross_entropy(pred_qa, qa)


## Training Question Answer Model

In [ ]:
import csv
import os
import pathlib
import random
import re

import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
def qa_train(model_cfg, dataset, device,  # model and datasets
             p_drop, n_epoch, batch_size, learning_rate,  # training hyper parameter
             save_step, model_path):  # saving model

    model = qa_model(**model_cfg, p_drop=p_drop).train().to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    dataldr = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    def save_model(md, step):
        torch.save(md.state_dict(), os.path.join(
            model_path, f"model-{step}.pt"))

    # Train loop
    step = 0
    for epoch in range(n_epoch):
        tqdm_dldr = tqdm(dataldr)
        
        avg_loss = 0
        for epoch_step, batch_data in enumerate(tqdm_dldr):
            optimizer.zero_grad()
            batch_data['article'] = batch_data['article'].numpy()
            batch_data["qa_answer"] = batch_data["qa_answer"].float()
            loss = model.loss_fn(
                document=torch.LongTensor(batch_data['article']).to(device),
                question=batch_data["question"].to(device),
                choice=batch_data["choice"].to(device),
                qa=batch_data["qa_answer"].to(device))
            loss.backward()
            optimizer.step()

            step += 1
            avg_loss += loss
            tqdm_dldr.set_description(
                f"epoch:{epoch}, loss:{avg_loss / (epoch_step+1):.04f}")

            if step % save_step == 0:
                save_model(model, step)

    save_model(model, step)


random_seed = 42
# Set random states for reproducibility
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
# Use cuda when possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Save training configuration
qa_model_path = os.path.join("exp", "qa")
pathlib.Path(qa_model_path).mkdir(parents=True, exist_ok=True)

#dataset = dataset_qa(
#    vocab_path=os.path.join("data", "vocab.json"),
#    qa_file=os.path.join("data", "SampleData_QA.json"),
#)

model_cfg = {
    "embedding_path": os.path.join("data", "embeddings.npy"),
    "d_emb": 300,
    "n_cls_layers": 2,
}

qa_train(
    model_cfg=model_cfg,
    dataset=dataset,
    model_path=qa_model_path,
    device=device,
    # Hyperparameters
    batch_size=3,
    learning_rate=1e-4,
    n_epoch=45,
    save_step=18,
    p_drop=0.0,
)



  0%|          | 0/232 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:103: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
epoch:44, loss:0.0472: 100%|██████████| 232/232 [00:07<00:00, 31.04it/s]


## Test on trained QA model

In [ ]:
from sklearn.metrics import accuracy_score
import csv
import os
import pathlib
import random
import re
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
def save_result(output_path: str, data: list, ckpt: int):
    output = [["id", "label"]] + \
        [[i+1, label] for i, label in enumerate(data)]
    csv.writer(open(os.path.join(
        output_path, f"decision_{ckpt}.csv"), 'w', newline='')).writerows(output)
def qa_test(model_cfg, dataset, device, batch_size,
            model_path, output_path):
    dataldr = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=False)

    # Load all checkpoints
    ckpts = sorted([
        (int(ckpt.group(1)), os.path.join(model_path, ckpt.group(0)))
        for ckpt in map(lambda f:re.match(r'model-(\d+).pt', f), os.listdir(model_path))
        if ckpt is not None
    ], key=lambda x: x[0])

    sttep=0

    for step, ckpt in ckpts:
        model = qa_model(**model_cfg, p_drop=0.0)
        model.load_state_dict(torch.load(ckpt))
        model = model.eval().to(device)
        sttep=sttep+1
        print(sttep,end=' ')
        answer = []
        preds = []
        for batch_data in tqdm(dataldr):
            answer += batch_data["qa_answer"].argmax(dim=-1).tolist()
            batch_data['article'] = batch_data['article'].numpy()
            batch_data["question"] = torch.tensor(batch_data["question"]).to(device).long()
            batch_data["choice"] = torch.tensor(batch_data["choice"]).to(device).long()
            pred_qa = model(
                document=torch.LongTensor(batch_data['article']).to(device),
                question=batch_data["question"].to(device),
                choice=batch_data["choice"].to(device))
            #print(pred_qa.argmax(dim=-1).tolist())
            #preds += pred_qa.argmax(dim=-1).tolist()
            preds.append(dic[int(pred_qa.argmax(dim=-1))])
            #print(f"\nstep {step} accuracy: {accuracy_score(answer, preds):.04f}", flush=True)
        save_result(output_path, preds, step)

dic = {0: 'A', 1: 'B', 2: 'C'}
qa_output_path = os.path.join("output", "qa")
pathlib.Path(qa_output_path).mkdir(parents=True, exist_ok=True)

print("\nevaluate on training set...", flush=True)

qa_test(
    model_cfg=model_cfg,
    dataset=dataset,
    model_path=qa_model_path,
    device=device,
    batch_size=1,
    output_path=qa_output_path,
)


evaluate on training set...


  0%|          | 0/695 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  2%|▏         | 14/695 [00:00<00:05, 133.64it/s]

1 

  2%|▏         | 14/695 [00:00<00:05, 134.56it/s]

2 

...

  2%|▏         | 14/695 [00:00<00:04, 139.55it/s]

579 

  2%|▏         | 14/695 [00:00<00:05, 135.90it/s]

580 

100%|██████████| 695/695 [00:05<00:00, 135.30it/s]


## Preditions

In [ ]:
qa_latest_result = sorted([
    (int(ckpt.group(1)), os.path.join(qa_output_path, ckpt.group(0)))
    for ckpt in map(lambda f:re.match(r'decision_(\d+).csv', f), os.listdir(qa_output_path))
    if ckpt is not None
], key=lambda x: x[0])[-1]

pd.read_csv(open(qa_latest_result[1]))

,id,label
0,1,C
1,2,A
2,3,C
3,4,C
4,5,C
...,...,...
690,691,B
691,692,C
692,693,B
693,694,C


In [ ]:
dff = pd.read_json(r'/content/data/Train_qa_ans.json')
dff.to_csv (r'output/decision.csv', index = None)

In [ ]:
with open('output/decision_10440.csv') as csvfile:
    rows = csv.reader(csvfile)
    i=0
    for row in rows:
        i=i+1
        print(row[1].strip(),i)

label 1
C 2
A 3
C 4
C 5
C 6
B 7
C 8
C 9
B 10
C 11
C 12
A 13
C 14
A 15
A 16
C 17
B 18
A 19
A 20
A 21
B 22
A 23
C 24
C 25
B 26
C 27
B 28
B 29
B 30
A 31
C 32
A 33
C 34
C 35
C 36
C 37
B 38
B 39
A 40
C 41
C 42
B 43
A 44
C 45
A 46
C 47
C 48
C 49
A 50
A 51
B 52
C 53
A 54
C 55
C 56
C 57
A 58
A 59
B 60
A 61
A 62
B 63
A 64
B 65
C 66
A 67
B 68
C 69
B 70
A 71
B 72
B 73
C 74
C 75
B 76
A 77
B 78
B 79
A 80
A 81
C 82
A 83
A 84
A 85
A 86
B 87
C 88
A 89
A 90
C 91
A 92
A 93
C 94
C 95
C 96
A 97
C 98
A 99
C 100
C 101
B 102
B 103
B 104
C 105
A 106
C 107
B 108
B 109
A 110
C 111
B 112
B 113
A 114
C 115
A 116
C 117
A 118
A 119
C 120
C 121
A 122
B 123
C 124
B 125
C 126
C 127
A 128
C 129
C 130
C 131
B 132
C 133
B 134
A 135
C 136
A 137
C 138
A 139
B 140
C 141
A 142
A 143
B 144
C 145
B 146
B 147
A 148
A 149
C 150
C 151
B 152
A 153
A 154
A 155
B 156
A 157
A 158
B 159
A 160
C 161
B 162
B 163
C 164
C 165
A 166
A 167
C 168
C 169
C 170
C 171
C 172
A 173
A 174
A 175
C 176
B 177
C 178
A 179
C 180
C 181
C 182
C 183
C 184


In [ ]:
import csv
ans_file = open('output/decision.csv')
decision_file = open('output/decision_10440.csv')
rows1 = csv.reader(ans_file)
rows2 = csv.reader(decision_file)
i1=0
i2=0
for row1,row2 in zip(rows1,rows2):
    print("正解:",row1[4].strip(),"算出:",row2[1].strip())
    if row1[4].strip()==row2[1].strip():
        i2=i2+1
    else:
        print("============Error item============",sep='')
    i1=i1+1
print(i1,i2,i2/i1)

正解: answer 算出: label
============Error item============
正解: C 算出: C
正解: A 算出: A
正解: C 算出: C
正解: C 算出: C
正解: C 算出: C
正解: B 算出: B
正解: C 算出: C
正解: C 算出: C
正解: B 算出: B
正解: C 算出: C
正解: Ｃ 算出: C
============Error item============
正解: Ａ 算出: A
============Error item============
正解: C 算出: C
正解: Ａ 算出: A
============Error item============
正解: A 算出: A
正解: Ｃ 算出: C
============Error item============
正解: Ｂ 算出: B
============Error item============
正解: A 算出: A
正解: A 算出: A
正解: A 算出: A
正解: B 算出: B
正解: Ａ 算出: A
============Error item============
正解: C 算出: C
正解: C 算出: C
正解: B 算出: B
正解: C 算出: C
正解: B 算出: B
正解: B 算出: B
正解: B 算出: B
正解: Ａ 算出: A
============Error item============
正解: C 算出: C
正解: Ａ 算出: A
============Error item============
正解: C 算出: C
正解: C 算出: C
正解: C 算出: C
正解: C 算出: C
正解: B 算出: B
正解: B 算出: B
正解: A 算出: A
正解: C 算出: C
正解: C 算出: C
正解: B 算出: B
正解: A 算出: A
正解: C 算出: C
正解: Ａ 算出: A
============Error item============
正解: C 算出: C
正解: C 算出: C
正解: C 算出: C
正解: A 算出: A
正解: A 算出: A
正解: Ｂ 算出: B
============Error